In [1]:
import pickle

In [2]:
data = pickle.load(open('./pickles/realtor_clean.p','rb'))
residual_model = pickle.load(open('./pickles/final_model.p','rb'))
location_model = pickle.load(open('./pickles/location_model.p','rb'))

GridSearchCV(cv=None, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': range(10, 100)},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)